# Task 1


In [10]:
from sqlite3.dbapi2 import Timestamp

import pandas as pd
import numpy as np

In [101]:
df = pd.read_parquet("data/spx_ivs_2023-02.parquet")
df["rate"] = np.log(df["underlyingforwardprice"] / df["underlyingprice"]) /  (30 / 365)
df["test normalized moneyness"] = np.log(df["forwardMoneyness"]) / (np.sqrt(30/365) * df['implVol'])

#df["loctimestamp"] = pd.to_datetime(df["loctimestamp"])
#df["loctimestamp"].rename("date")

grouped_by_day = {date : group for date,group in df.groupby('loctimestamp')}

# strike price
K = df["strike"].copy()

# underlying price at time t
S_t = df["underlyingprice"].copy()

# call price C(t, tau, K) depending of an underlying asset with price S(t), date to maturity tau, strike price K
C = df[df["putcall"] == "C"]["implPrice"].copy()

# put price C(t, tau, K) depending of an underlying asset with price S(t), date to maturity tau, strike price K
P = df[df["putcall"] == "P"]["implPrice"].copy()

df

,Symbol,loctimestamp,putcall,daystomaturity,implVol,implPrice,strike,forwardMoneyness,normalizedMoneyness,underlyingprice,underlyingforwardprice,rate,test normalized moneyness
0,^SPX,2023-02-01,P,30,0.546362,0.162479,2572.16,0.620158,-10.00,4138.45,4147.58,0.026812,-3.050241
1,^SPX,2023-02-01,P,30,0.537310,0.176469,2603.06,0.627610,-9.75,4138.45,4147.58,0.026812,-3.024087
2,^SPX,2023-02-01,P,30,0.528999,0.195240,2634.34,0.635152,-9.50,4138.45,4147.58,0.026812,-2.992833
3,^SPX,2023-02-01,P,30,0.520193,0.214198,2666.00,0.642784,-9.25,4138.45,4147.58,0.026812,-2.963405
4,^SPX,2023-02-01,P,30,0.510792,0.232522,2698.03,0.650507,-9.00,4138.45,4147.58,0.026812,-2.936388
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1078,^SPX,2023-02-28,C,30,0.171093,0.032910,4672.92,1.172590,3.00,3974.66,3985.14,0.032038,3.245918
1079,^SPX,2023-02-28,C,30,0.179106,0.022557,4735.33,1.188250,3.25,3974.66,3985.14,0.032038,3.359066
1080,^SPX,2023-02-28,C,30,0.186005,0.014515,4798.57,1.204120,3.50,3974.66,3985.14,0.032038,3.483274
1081,^SPX,2023-02-28,C,30,0.191016,0.008167,4862.66,1.220200,3.75,3974.66,3985.14,0.032038,3.634138


In [68]:
v_integral = (
    np.trapezoid(
        (2 * (1 - np.log(K / S_t)) / np.square(K) * C)[df["loctimestamp"] == pd.Timestamp("2023-02-01")].dropna(),
        K[df["putcall"] == "C"][df["loctimestamp"] == pd.Timestamp("2023-02-01")]
    ) +
    np.trapezoid(
        (2 * (1 + np.log(S_t / K)) / np.square(K) * P)[df["loctimestamp"] == pd.Timestamp("2023-02-01")].dropna(),
        K[df["putcall"] == "P"][df["loctimestamp"] == pd.Timestamp("2023-02-01")]
    )
)
v_integral


np.float64(0.002852815975177324)

In [53]:
K[df["putcall"] == "C"][df["loctimestamp"] == pd.Timestamp("2023-02-01")]''#filter test
(2 * (1 - np.log(K / S_t)) / np.square(K) * C)[df["loctimestamp"] == pd.Timestamp("2023-02-01")].dropna()

40    9.134897e-06
41    6.046839e-06
42    3.751367e-06
43    2.149516e-06
44    1.138460e-06
45    5.673463e-07
46    2.748220e-07
47    1.366573e-07
48    6.908388e-08
49    3.685803e-08
50    2.111116e-08
51    1.289253e-08
52    8.400070e-09
53    5.910084e-09
54    3.667400e-09
55    2.169096e-09
56    1.474381e-09
dtype: float64

In [115]:
df_firstday = df[df["loctimestamp"] == pd.Timestamp("2023-02-02")].copy()
df_firstday

,Symbol,loctimestamp,putcall,daystomaturity,implVol,implPrice,strike,forwardMoneyness,normalizedMoneyness,underlyingprice,underlyingforwardprice,rate,test normalized moneyness
57,^SPX,2023-02-02,P,30,0.573596,0.142354,2506.60,0.600861,-10.00,4163.94,4171.68,0.022595,-3.097644
58,^SPX,2023-02-02,P,30,0.565357,0.159296,2538.73,0.608561,-9.75,4163.94,4171.68,0.022595,-3.064224
59,^SPX,2023-02-02,P,30,0.557467,0.179992,2571.26,0.616361,-9.50,4163.94,4171.68,0.022595,-3.027906
60,^SPX,2023-02-02,P,30,0.548773,0.200431,2604.22,0.624260,-9.25,4163.94,4171.68,0.022595,-2.994936
61,^SPX,2023-02-02,P,30,0.539460,0.220851,2637.59,0.632261,-9.00,4163.94,4171.68,0.022595,-2.964295
62,^SPX,2023-02-02,P,30,0.530797,0.247349,2671.40,0.640364,-8.75,4163.94,4171.68,0.022595,-2.928991
63,^SPX,2023-02-02,P,30,0.521878,0.276287,2705.63,0.648571,-8.50,4163.94,4171.68,0.022595,-2.893933
64,^SPX,2023-02-02,P,30,0.511873,0.302492,2740.31,0.656883,-8.25,4163.94,4171.68,0.022595,-2.863721
65,^SPX,2023-02-02,P,30,0.501880,0.332228,2775.43,0.665302,-8.00,4163.94,4171.68,0.022595,-2.832231
66,^SPX,2023-02-02,P,30,0.493111,0.375556,2811.00,0.673829,-7.75,4163.94,4171.68,0.022595,-2.792512


In [116]:
S_t = df_firstday["underlyingprice"].copy()
C = df_firstday[df_firstday["putcall"] == "C"]["implPrice"].copy()
P = df_firstday[df_firstday["putcall"] == "P"]["implPrice"].copy()

puts = (2 * (1 + np.log(S_t / K)) / np.square(K) * P).dropna()
calls = (2 * (1 - np.log(K / S_t)) / np.square(K) * C).dropna()

x = K[df["loctimestamp"] == pd.Timestamp("2023-02-02")]
y = pd.concat([puts, calls])

df_total = pd.concat([x, y], axis=1)
df_total.columns = ['x', 'y']
df_total["x_shifted"] = df_total["x"].shift()
df_total["y_shifted"] = df_total["y"].shift()
df_total.dropna()

integral = ((df_total["x"] - df_total["x_shifted"]) * (df_total["y"] + df_total["y_shifted"]) / 2).sum()
integral


np.float64(0.003307551209547153)